Import library yang akan dibutuhkan

In [2]:
import pandas as pd
import numpy as np
import streamlit as st
import joblib

Read dataset dan melihat detail dataset

In [3]:
df = pd.read_csv('dataset.csv')
# station_id = 96735
# df = df[df['station_id'] == station_id]
df.head()

,date,Tn,Tx,Tavg,RH_avg,RR,ss,ff_x,ddd_x,ff_avg,ddd_car,station_id
0,2018-01-01,24.0,34.0,28.1,80.0,21.3,8.0,6.0,240.0,5.0,W,96733
1,2018-01-02,24.0,34.0,29.7,75.0,NaN,6.9,5.0,250.0,6.0,W,96733
2,2018-01-03,26.0,34.8,28.3,78.0,NaN,4.3,5.0,280.0,7.0,W,96733
3,2018-01-04,26.0,33.8,29.0,76.0,NaN,4.8,5.0,230.0,6.0,W,96733
4,2018-01-05,26.0,30.8,27.8,72.0,0.3,1.8,7.0,250.0,10.0,W,96733


In [4]:
df.describe()

,Tn,Tx,Tavg,RH_avg,RR,ss,ff_x,ddd_x,ff_avg,station_id
count,31374.000000,31601.000000,31075.000000,31054.000000,24109.000000,28827.000000,32543.000000,32536.000000,32543.000000,32564.000000
mean,23.366568,31.586504,27.009445,78.817254,7.610610,6.177032,4.641705,181.797855,1.903482,96845.308807
std,2.736772,3.823300,2.279563,43.034775,16.558978,5.157914,2.091569,106.710820,1.120372,86.203201
min,2.400000,3.600000,17.900000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96733.000000
25%,22.000000,30.200000,26.000000,73.000000,0.000000,4.000000,3.000000,90.000000,1.000000,96753.000000
50%,24.000000,32.000000,27.500000,79.000000,0.000000,6.600000,4.000000,170.000000,2.000000,96835.000000
75%,25.000000,33.200000,28.600000,84.000000,7.000000,8.600000,6.000000,280.000000,2.000000,96937.000000
max,36.000000,334.000000,34.300000,7520.000000,277.500000,705.000000,26.000000,360.000000,11.000000,96987.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32564 entries, 0 to 32563
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        32564 non-null  object 
 1   Tn          31374 non-null  float64
 2   Tx          31601 non-null  float64
 3   Tavg        31075 non-null  float64
 4   RH_avg      31054 non-null  float64
 5   RR          24109 non-null  float64
 6   ss          28827 non-null  float64
 7   ff_x        32543 non-null  float64
 8   ddd_x       32536 non-null  float64
 9   ff_avg      32543 non-null  float64
 10  ddd_car     32564 non-null  object 
 11  station_id  32564 non-null  int64  
dtypes: float64(9), int64(1), object(2)
memory usage: 3.0+ MB


In [6]:
df.isna().sum()

date             0
Tn            1190
Tx             963
Tavg          1489
RH_avg        1510
RR            8455
ss            3737
ff_x            21
ddd_x           28
ff_avg          21
ddd_car          0
station_id       0
dtype: int64

Mengisi data kosong yang ada di dalam dataset

In [7]:
def fillna_groupby_mean(df, columns):
    for column in columns:
        df[column] = df.groupby('station_id')[column].transform(lambda x: x.fillna(x.mean()))


def fillna_groupby_mode(df, columns):
    for column in columns:
        df[column] = df.groupby('station_id')[column].transform(lambda x: x.fillna(x.mode().iloc[0]))

fillna_groupby_mean(df, ['Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg'])
fillna_groupby_mode(df, ['ddd_car'])

In [8]:
df.isna().sum()

date             0
Tn               0
Tx               0
Tavg             0
RH_avg        1067
RR               0
ss            1409
ff_x             0
ddd_x            0
ff_avg           0
ddd_car          0
station_id       0
dtype: int64

Karena masih ada data kosong detelah diisi, dataset kosong sisanya akan dihapus

In [9]:
df.dropna(inplace=True)

Encode / merubah tipe data object menjadi data numerik menggunakan label encoder

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['ddd_car']= label_encoder.fit_transform(df['ddd_car'])

Mengecek apakah ada outlier / tidak pada sebaran data

In [11]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3-Q1

    outliers = df[(df < (Q1-1.5*IQR)) | (df > (Q3+1.5*IQR))]
    return outliers

def print_outlier(df):
    for i in df.columns:
        if i not in ['date']:
            print(f"Number of Outliers in {i}: {len(check_outlier(df[i]))}")

print_outlier(df)

Number of Outliers in Tn: 1258
Number of Outliers in Tx: 1053
Number of Outliers in Tavg: 1290
Number of Outliers in RH_avg: 253
Number of Outliers in RR: 2218
Number of Outliers in ss: 6
Number of Outliers in ff_x: 375
Number of Outliers in ddd_x: 0
Number of Outliers in ff_avg: 1923
Number of Outliers in ddd_car: 0
Number of Outliers in station_id: 0


Menghapus outlier pada sebaran data

In [12]:
def handle_outlier(df):
    for i in df.columns:
        if i not in ['date']:
            Q1 = df[i].quantile(0.25)
            Q3 = df[i].quantile(0.75)
            IQR = Q3-Q1

            lower_bound = Q1 - 1.5*IQR
            upper_bound = Q3 + 1.5*IQR

            df[i] = np.where(df[i] < lower_bound, lower_bound, df[i])
            df[i] = np.where(df[i] > upper_bound, upper_bound, df[i])

            print(f'Column {i} has been handled')
    print('It is Done!')

handle_outlier(df)

Column Tn has been handled
Column Tx has been handled
Column Tavg has been handled
Column RH_avg has been handled
Column RR has been handled
Column ss has been handled
Column ff_x has been handled
Column ddd_x has been handled
Column ff_avg has been handled
Column ddd_car has been handled
Column station_id has been handled
It is Done!


Memilih fitur yang akan digunakan oleh model

In [13]:
features = ['Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 
            'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']
X = df[features]

Melakukan normalisasi menggunakan standard scaler

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Melakukan reduksi dimensi pada data

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

Mencari jumlah n cluster optimal menggunakan elbow method

In [16]:
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
def optimise_k_means(df, max_k):
    means = []
    inertias = []
    
    for k in range(1, max_k):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(df)
        
        means.append(k)
        inertias.append(kmeans.inertia_)
        
    fig = plt.subplots(figsize=(10,5))
    plt.plot(means, inertias, 'o-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()

optimise_k_means(df.drop(['date','station_id'], axis = 1), 10)

C:\Users\Rave\AppData\Local\Temp\ipykernel_27024\697528764.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Mencari jumlah n cluster optimal menggunakan silhouette method

In [17]:
# Menentukan rentang jumlah cluster yang akan diuji
from sklearn.metrics import silhouette_score
range_n_clusters = list(range(2, 7))

# Menyimpan silhouette scores untuk setiap jumlah cluster
silhouette_scores = []

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X_pca)
    
    silhouette_avg = silhouette_score(X_pca, labels)
    silhouette_scores.append(silhouette_avg)
    print(f'Jumlah cluster: {n_clusters} | Silhouette Score: {silhouette_avg}')

Jumlah cluster: 2 | Silhouette Score: 0.3210568224689188
Jumlah cluster: 3 | Silhouette Score: 0.29393191931925655
Jumlah cluster: 4 | Silhouette Score: 0.3052004663881294
Jumlah cluster: 5 | Silhouette Score: 0.2653099476032021
Jumlah cluster: 6 | Silhouette Score: 0.283100503112427


Mengaplikasikan model yang akan digunakan

In [18]:
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D

# KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
df['KMeans_Cluster'] = kmeans.fit_predict(X_pca)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2)
df['DBSCAN_Cluster'] = dbscan.fit_predict(X_pca)

# Spectral Clustering
spectral = SpectralClustering(n_clusters=4, affinity='nearest_neighbors', random_state=42)
df['Spectral_Cluster'] = spectral.fit_predict(X_pca)

# Gaussian Mixture Model
gmm = GaussianMixture(n_components=4, random_state=42)
df['GMM_Cluster'] = gmm.fit_predict(X_pca)

# Plotting fungsi
def plot_3d(X, labels, title):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=labels, cmap='viridis')
    legend1 = ax.legend(*scatter.legend_elements(), title="Cluster")
    ax.add_artist(legend1)
    ax.set_title(title)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')
    ax.set_zlabel('PCA Component 3')
    plt.show()

# Plotting hasil clustering
plot_3d(X_pca, df['KMeans_Cluster'], 'KMeans Clustering')
plot_3d(X_pca, df['DBSCAN_Cluster'], 'DBSCAN Clustering')
plot_3d(X_pca, df['Spectral_Cluster'], 'Spectral Clustering')
plot_3d(X_pca, df['GMM_Cluster'], 'Gaussian Mixture Model Clustering')

C:\Users\Rave\AppData\Local\Temp\ipykernel_27024\3713045052.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Melakukan evaluasi kinerja model menggunakan silhouette score, davies bouldin score, dan calinski harabarz score

In [19]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Evaluasi clustering
def evaluate_clustering(X, labels, method_name):
    # Pastikan semua label adalah numerik
    unique_labels = np.unique(labels)
    if len(unique_labels) > 1:
        silhouette_avg = silhouette_score(X, labels)
        davies_bouldin_avg = davies_bouldin_score(X, labels)
        calinski_harabasz_avg = calinski_harabasz_score(X, labels)
        print(f'{method_name} Clustering:')
        print(f'Silhouette Score: {silhouette_avg:.3f}')
        print(f'Davies-Bouldin Index: {davies_bouldin_avg:.3f}')
        print(f'Calinski-Harabasz Index: {calinski_harabasz_avg:.3f}')
    else:
        print(f'{method_name} Clustering: Only one cluster found')
    print('---------------------------------')

evaluate_clustering(X_pca, df['KMeans_Cluster'], 'KMeans')
evaluate_clustering(X_pca, df['DBSCAN_Cluster'], 'DBSCAN')
evaluate_clustering(X_pca, df['Spectral_Cluster'], 'Spectral')
evaluate_clustering(X_pca, df['GMM_Cluster'], 'GMM')

KMeans Clustering:
Silhouette Score: 0.305
Davies-Bouldin Index: 1.052
Calinski-Harabasz Index: 15492.785
---------------------------------
DBSCAN Clustering:
Silhouette Score: -0.064
Davies-Bouldin Index: 1.844
Calinski-Harabasz Index: 6.449
---------------------------------
Spectral Clustering:
Silhouette Score: 0.302
Davies-Bouldin Index: 1.081
Calinski-Harabasz Index: 15200.630
---------------------------------
GMM Clustering:
Silhouette Score: 0.285
Davies-Bouldin Index: 1.056
Calinski-Harabasz Index: 14010.577
---------------------------------
